In [1]:
import numpy as np
import pandas as pd
import sklearn
import mlflow

In [2]:
mlflow.set_tracking_uri('http://localhost:5000')

In [3]:
train_dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')
validation_dataset = pd.read_csv('validation.csv')

In [4]:
train_dataset

,Preprocessed_text,Spam
0,analyst candidate mitra mujica mitra mujica ac...,0
1,approval overdue access request paul thomas en...,0
2,giuseppe cell phone stinson problem vince stin...,0
3,2000 expenses know deadline 2000 invoices ap f...,0
4,houston trip extended rental apartment 20 th n...,0
...,...,...
4004,howard lawrence vince hey vince picture howard...,0
4005,custom logos identities us thinking breathing ...,1
4006,reactions september live line reactions latest...,0
4007,vince kaminski discussion notes enterprise wid...,0


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

In [6]:
X_train = tfidf.fit_transform(train_dataset['Preprocessed_text']).toarray()
y_train = train_dataset['Spam']

In [7]:
X_test = tfidf.transform(test_dataset['Preprocessed_text']).toarray()
y_test = test_dataset['Spam']


In [8]:
X_validation = tfidf.transform(validation_dataset['Preprocessed_text']).toarray()
y_validation = validation_dataset['Spam']

In [21]:
from sklearn.metrics import auc, precision_score, recall_score, confusion_matrix, precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid
from random import seed

import warnings
warnings.filterwarnings("ignore")

param_grid = {
    'max_depth': [2,4,6,8,10,15,20, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 7, 10]
}

Parameters
MetricsParameters
Metrics

with mlflow.start_run(run_name= 'Decision Tree'):
    best_auc = 0
    best_params = None
    for params in ParameterGrid(param_grid):
        seed(564)
        clf = DecisionTreeClassifier(**params)
        clf.fit(X_train, y_train)

        mlflow.sklearn.log_model(clf, f"[ {params['max_depth']}, {params['min_samples_split']}, {params['min_samples_leaf']} ]")
        precision, recall, _ = precision_recall_curve(y_validation, clf.predict_proba(X_validation)[:,1])
        sorted_indices = np.argsort(precision)
        precision = precision[sorted_indices]
        recall = recall[sorted_indices]
        validation_auc = auc(precision, recall)
        if validation_auc > best_auc:
            best_auc = validation_auc
            best_params = params
            
        
    mlflow.log_params(best_params)
    mlflow.log_metric("validation_auc", best_auc)

    seed(564)

    best_model_DT = DecisionTreeClassifier(**best_params)
    best_model_DT.fit(X_train, y_train)

    mlflow.sklearn.log_model(best_model_DT, "Best Decision Tree model")
        

KeyboardInterrupt: 

In [16]:
for i in **params:
    print(i)

SyntaxError: invalid syntax (1091596126.py, line 1)